In [19]:
import ray
from time import time 
from matplotlib import pyplot as plt
from numpy.linalg import norm
from Quantum_functions import *
import numpy as np
from QuantumPFunctions import *
from numpy import allclose


In [3]:
gb = 1024*1024*1024

from qutip import *
# Start Ray.
ray.init()
#ray.init(memory =14*gb, object_store_memory = 12*gb)

2020-04-02 19:09:38,114	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-04-02 19:09:38,116	INFO resource_spec.py:212 -- Starting Ray with 7.91 GiB memory available for workers and up to 3.97 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-02 19:09:38,424	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.97',
 'redis_address': '192.168.1.97:40208',
 'object_store_address': '/tmp/ray/session_2020-04-02_19-09-38_113903_23406/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-02_19-09-38_113903_23406/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-02_19-09-38_113903_23406'}

In [ ]:
n = 4
processors = 4
state1 = tensor([basis(2,0)]*n)

alpha1 = Heisenberg1dRingGen(1,1,1,n)
alpha0 = lambda n,m,i,j: 0
beta1 = lambda n,i : [0,0,1][i]
beta0 = lambda n,i :0

self_interaction = hamiltonian(alpha1,beta0,n)
self_interaction = self_interaction/self_interaction.norm()
external_interaction = hamiltonian(alpha0,beta1,n)
external_interaction = external_interaction/external_interaction.norm()


perturbation: Qobj = make_hermitian(rand_unitary(2**n, dims = self_interaction.dims))
perturbation = perturbation/perturbation.norm()
epsilon = 0.3

H1 = epsilon*self_interaction + external_interaction + epsilon**2*perturbation
H2 = external_interaction
H3 = external_interaction + epsilon*perturbation

### Testing Trace vs Energy Compare

In [ ]:
start = time()
xs,ys = energy_trace_comp_2d(H1,1,1)
end = time()

single_t = end-start

plt.scatter(xs,ys,s=1)
plt.show()
print(f"Finished executing completed in {end-start}s")

In [ ]:
start = time()
xs,ys = energy_trace_compare_p(H1,1,1,proc =processors)
end = time()

mulit_t  = end-start
print(len(xs))

plt.scatter(xs,ys,s=1)
plt.show()

print(f"Finished executing completed in {end-start}s")


In [ ]:
print(f"System obtains a speedup of {single_t/mulit_t} using {processors} processors" )

### Testing simulation compare

In [ ]:
n=6
state1 = tensor([basis(2,0)]*n)
processors = 4
beta0 = lambda n,m: 0
alpha1 = Heisenberg1dRingGen(-1,1,1,n)

H1 = hamiltonian(alpha1,beta0, n)
H1=H1/H1.norm()

assert(H1.isherm)

In [ ]:
start = time()
equilibration_analyser(H1,state1,1e5,200)
end=time()
print(f"Finished executing completed in {end-start}s")
sim_s = end-start

In [ ]:
start = time()
energys,states = H1.eigenstates()
print("Found states")
equilibration_analyser_p(energys,states,state1,0,1e5,200,Proc=4)
end=time()
print(f"Finished executing completed in {end-start}s")
sim_m = end-start

In [ ]:
print(f"System obtains a speedup of {sim_s/sim_m} using {processors} processors" )

In [ ]:
energys,states = H1.eigenstates()

In [ ]:
print(states[0])
print(ket2dm(states[0]))

### Testing Equilibration Density Operator Function

In [44]:
n = 11
processors = 4
initial_state = tensor([basis(2,0)]*n)

alpha1 = Heisenberg1dRingGen(1,1,1,n)
alpha0 = lambda n,m,i,j: 0
beta1 = lambda n,i : [0,0,1][i]
beta0 = lambda n,i :0

self_interaction = hamiltonian(alpha1,beta0,n)
self_interaction = self_interaction/self_interaction.norm()
external_interaction = hamiltonian(alpha0,beta1,n)
external_interaction = external_interaction/external_interaction.norm()


perturbation: Qobj = make_hermitian(rand_unitary(2**n, dims = self_interaction.dims))
perturbation = perturbation/perturbation.norm()
epsilon = 0.3

H1 = epsilon*self_interaction + external_interaction + epsilon**2*perturbation
H2 = external_interaction
H3 = external_interaction + epsilon*perturbation
H4 = external_interaction + epsilon*self_interaction

In [45]:
start = time()
energys,states= H3.eigenstates()
coef = [initial_state.overlap(estate) for estate in states]
print("Finished single stuff")
eq_state = get_equilibrated_dens_op_P(states,coef,n,proc=processors)
end =time()

print("Finished PARRALELL")

start2 =time()
eq_old = get_equilibrated_dens_op(H3,initial_state)
end2 = time()


old_t = end2-start2
new_t = end-start

print(allclose(eq_state,eq_old))

print(f"Non parallell completed in {old_t:.3f} new paralell with {processors} processors completed in {new_t:.3f} with speed up {(old_t)/(new_t)}")

Finished single stuff
Finished PARRALELL
True
Non parallell completed in 313.069 new paralell with 4 processors completed in 106.716 with speed up 2.933652546571499
